# Student Admission

In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
data = pd.read_csv('https://stats.idre.ucla.edu/stat/data/binary.csv')

In [4]:
print(data.head())

   admit  gre   gpa  rank
0      0  380  3.61     3
1      1  660  3.67     3
2      1  800  4.00     1
3      1  640  3.19     4
4      0  520  2.93     4


In [15]:
X = data.iloc[:,[1,2,3]]
y = data.iloc[:,[0]]
X, y

(     gre   gpa  rank
 0    380  3.61     3
 1    660  3.67     3
 2    800  4.00     1
 3    640  3.19     4
 4    520  2.93     4
 ..   ...   ...   ...
 395  620  4.00     2
 396  560  3.04     3
 397  460  2.63     2
 398  700  3.65     2
 399  600  3.89     3
 
 [400 rows x 3 columns],      admit
 0        0
 1        1
 2        1
 3        1
 4        0
 ..     ...
 395      0
 396      0
 397      0
 398      0
 399      0
 
 [400 rows x 1 columns])

In [19]:
X_train, y_train, X_test, y_test = train_test_split(X,y, test_size=0.3, random_state=42)
X_train.head()

,gre,gpa,rank
157,480,3.58,1
109,480,3.45,2
17,360,2.56,3
347,580,3.80,2
24,760,3.35,2


In [20]:
n_features = 4
n_labels = 1

features = tf.placeholder(tf.float32)
labels = tf.placeholder(tf.float32)

w = tf.Variable(tf.truncated_normal((n_features, n_labels)))
b = tf.Variable(tf.zeros(n_labels))

linear = tf.add(tf.matmul(features, w),b)
init = tf.global_variables_initializer()

In [21]:
with tf.Session() as sess:
    sess.run(init)
    prediction = tf.nn.softmax(linear)

    cross_entropy = -tf.reduce_sum(labels*tf.log(prediction), reduction_indices=1)

    loss = tf.reduce_mean(cross_entropy)

    optim = tf.train.AdagradOptimizer(0.08).minimize(loss)
    _, l = sess.run([optim, loss],
                    feed_dict={features: X_train, labels: y_train})

print(f'Loss : {l}')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: In[0] mismatch In[1] shape: 3 vs. 4: [280,3] [4,1] 0 0
	 [[node MatMul (defined at c:\program files\python37\lib\site-packages\tensorflow_core\python\framework\ops.py:1762) ]]
	 [[Mean/_19]]
  (1) Invalid argument: In[0] mismatch In[1] shape: 3 vs. 4: [280,3] [4,1] 0 0
	 [[node MatMul (defined at c:\program files\python37\lib\site-packages\tensorflow_core\python\framework\ops.py:1762) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'MatMul':
  File "c:\program files\python37\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\program files\python37\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\program files\python37\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\program files\python37\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    